In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import glob

print(datetime.datetime.now())
os.getcwd()

2020-01-16 15:54:33.626084


'/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1'

In [2]:
def recursive_file_gen(root_path):
    for root, dirs, files in os.walk(root_path):
        for file in files:
            yield os.path.join(root,file)
            
date_last_Saturday=datetime.date(2020,1,11)
dates_3_months_earlier=date_last_Saturday-datetime.timedelta(days=3*4*7) # >= to include 13 weeks 

list_daily_sales=list(recursive_file_gen("/home/jian/BigLots/"))
list_daily_sales=[x for x in list_daily_sales if "daily" in x.lower() and "/MediaStorm_" in x]
list_daily_sales=[x for x in list_daily_sales if x.split("/MediaStorm_")[1][:10]>=str(dates_3_months_earlier)]
print(len(list_daily_sales))

13


In [3]:
list_daily_sales.sort()
print(list_daily_sales[0])
print(list_daily_sales[-1])


/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-10-19/MediaStormDailySales20191022-111240-145.txt
/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-01-11/MediaStormDailySales20200114-115009-140.txt


In [4]:
df_product_taxonomy=pd.read_csv("/home/jian/BigLots/static_files/ProductTaxonomy/MediaStormProductTaxonomy20200101-135600-916.txt",
                               dtype=str,sep="|",usecols=['division_id','class_code_id','subclass_id']).drop_duplicates()
print(df_product_taxonomy.shape)
print(df_product_taxonomy[['class_code_id','subclass_id']].drop_duplicates().shape)
df_product_taxonomy.head(2)

(2875, 3)
(2875, 2)


,division_id,class_code_id,subclass_id
0,1,11001,2
1,1,11001,4


In [5]:
df_total_sales_by_store_division_rewards=pd.DataFrame()
i_counter=0
for file in list_daily_sales:
    df=pd.read_table(file,dtype=str,sep="|",
                    usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed','class_code_id','subclass_id','item_transaction_amt'])
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df=df.rename(columns={"item_transaction_amt":"sales"})
    print(df['transaction_dt'].min(),df['transaction_dt'].max())
    df['rewards_group']=np.where(pd.isnull(df['customer_id_hashed']),"Non_Rewards","Rewards")
    df=pd.merge(df,df_product_taxonomy,on=['class_code_id','subclass_id'],how="left")
    df=df.groupby(['location_id','rewards_group','division_id'])['sales'].sum().to_frame().reset_index()
    
    df_total_sales_by_store_division_rewards=df_total_sales_by_store_division_rewards.append(df)
    i_counter+=1
    print(i_counter,datetime.datetime.now())
    
    

2019-10-13 2019-10-19
1 2020-01-16 15:57:39.229082
2019-10-20 2019-10-26
2 2020-01-16 15:58:58.346469
2019-10-27 2019-11-02
3 2020-01-16 16:00:31.924309
2019-11-03 2019-11-09
4 2020-01-16 16:01:19.635899
2019-11-10 2019-11-16
5 2020-01-16 16:02:13.131970
2019-11-17 2019-11-23
6 2020-01-16 16:03:16.948018
2019-11-24 2019-11-30
7 2020-01-16 16:05:49.193484
2019-12-01 2019-12-07
8 2020-01-16 16:07:17.729076
2019-12-08 2019-12-14
9 2020-01-16 16:09:15.811161
2019-12-15 2019-12-21
10 2020-01-16 16:11:42.252703
2019-12-22 2019-12-28
11 2020-01-16 16:12:50.011512
2019-12-29 2020-01-04
12 2020-01-16 16:13:40.685366
2020-01-05 2020-01-11
13 2020-01-16 16:14:22.931182


In [6]:
print(df_total_sales_by_store_division_rewards.shape)
df_total_sales_by_store_division_rewards.head(4)

(279118, 4)


,location_id,rewards_group,division_id,sales
0,1,Non_Rewards,1,4084.92
1,1,Non_Rewards,2,4095.71
2,1,Non_Rewards,3,954.21
3,1,Non_Rewards,4,4235.46


In [7]:
df_total_sales_by_store_division_rewards.shape

(279118, 4)

In [21]:
df_total_sales_by_store_division_rewards[df_total_sales_by_store_division_rewards['division_id']=="9"]['sales'].sum()

396457743.9899999

In [22]:
df_total_sales_by_store_division_rewards['sales'].sum()

1637192275.1499987

In [24]:
396457743.9899999/1637192275.1499987

0.24215710641175403

In [29]:
df_total_sales=df_total_sales_by_store_division_rewards.groupby(["location_id",'rewards_group'])['sales'].sum().to_frame().reset_index()
df_total_sales=df_total_sales.rename(columns={"sales":"total_sales"})
df_total_sales=df_total_sales.pivot_table(index="location_id",columns="rewards_group",values="total_sales").reset_index()
df_total_sales=df_total_sales.rename(columns={"Non_Rewards":"Non_Rewards_total_sales"})
df_total_sales=df_total_sales.rename(columns={"Rewards":"Rewards_total_sales"})
df_total_sales['total_sales']=df_total_sales['Non_Rewards_total_sales']+df_total_sales['Rewards_total_sales']

df_furniture_sales=df_total_sales_by_store_division_rewards[df_total_sales_by_store_division_rewards['division_id']=="9"]
df_furniture_sales=df_furniture_sales.groupby(["location_id",'rewards_group'])['sales'].sum().to_frame().reset_index()
df_furniture_sales=df_furniture_sales.rename(columns={"sales":"furniture_sales"})
df_furniture_sales=df_furniture_sales.pivot_table(index="location_id",columns="rewards_group",values="furniture_sales").reset_index()
df_furniture_sales=df_furniture_sales.rename(columns={"Non_Rewards":"Non_Rewards_furniture_sales"})
df_furniture_sales=df_furniture_sales.rename(columns={"Rewards":"Rewards_furniture_sales"})
df_furniture_sales['furniture_sales']=df_furniture_sales['Non_Rewards_furniture_sales']+df_furniture_sales['Rewards_furniture_sales']

df_output=pd.merge(df_total_sales,df_furniture_sales,on="location_id",how="outer").fillna(0)


In [31]:
df_output['location_id']=df_output['location_id'].astype(int)
df_output=df_output.sort_values("location_id")
# df_output['store_type']=np.where(df_output['location_id']==6990,"Online","Instore")
df_output=df_output[df_output['location_id']!=6990]

In [32]:
df_output['total_fur_share']=df_output['furniture_sales']/df_output['total_sales']
df_output=df_output.sort_values("total_fur_share",ascending=False)



df_output['rank']=[1+x for x in range(len(df_output))]

df_output['store_furniture_group']=np.where(df_output['rank']>=len(df_output)*0.75,"Low","High")

In [33]:
writer=pd.ExcelWriter("./BL_output_store_type_furniture_3_months_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_output.to_excel(writer,"store_type_recent_3_months",index=False)
writer.save()



In [34]:
df_product_taxonomy.head(2)

,division_id,class_code_id,subclass_id
0,1,11001,2
1,1,11001,4


In [25]:
qc_df=pd.read_table(file,dtype=str,sep="|",
                    usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed','class_code_id','subclass_id','item_transaction_amt'])
qc_df['item_transaction_amt']=qc_df['item_transaction_amt'].astype(float)
qc_df=pd.merge(qc_df,df_product_taxonomy,on=['class_code_id','subclass_id'],how="left")
qc_df=qc_df.groupby("division_id")['item_transaction_amt'].sum().to_frame().reset_index()
qc_df

,division_id,item_transaction_amt
0,1,1.195827e+07
1,10,2.722203e+04
2,2,1.303302e+07
3,3,5.355936e+06
4,4,1.416551e+07
5,5,5.124858e+06
6,6,5.710518e+06
7,9,1.901121e+07


In [31]:
qc_df.loc[qc_df['division_id']=="9",'item_transaction_amt'].values[0]/qc_df['item_transaction_amt'].sum()

0.2555732682280726

In [14]:
df_output.shape

(1426, 10)

In [15]:
df_output

rewards_group,location_id,Non_Rewards_total_sales,Rewards_total_sales,total_sales,Non_Rewards_furniture_sales,Rewards_furniture_sales,furniture_sales,total_fur_share,rank,store_furniture_group
615,371,15895.01,37849.90,53744.91,856.600000,2432.106667,3288.706667,0.061191,1,High
372,175,12611.88,19807.47,32419.35,646.150000,774.020000,1420.170000,0.043806,2,High
186,1392,358571.32,1013726.08,1372297.40,8947.279231,40246.630769,49193.910000,0.035848,3,High
607,344,26670.60,36458.30,63128.90,732.393333,1447.023333,2179.416667,0.034523,4,High
1345,5388,498476.58,1008650.36,1507126.94,16726.256154,34858.624615,51584.880769,0.034227,5,High
...,...,...,...,...,...,...,...,...,...,...
731,4225,680386.87,1376007.22,2056394.09,-17.614444,39.760833,22.146389,0.000011,1422,Low
643,4046,363005.54,555443.90,918449.44,29.990000,-28.993333,0.996667,0.000001,1423,Low
764,4286,608901.04,778742.29,1387643.33,-10.527500,-0.248000,-10.775500,-0.000008,1424,Low
653,4068,514050.59,619995.43,1134046.02,-46.500000,20.971667,-25.528333,-0.000023,1425,Low


In [17]:
df['sales'].sum()

74386545.82999991

In [19]:
df[df['division_id']=="9"]['sales'].sum()

19011212.62999998

In [20]:
19011212.62999998/74386545.82999991

0.2555732682284705